In [2]:
import scanpy as sc
import numpy as np
import csv
import pickle
import matplotlib
import math
from itertools import combinations
import itertools
import pandas as pd
import matplotlib
from pyvis.network import Network
import networkx as nx
from networkx.algorithms import bipartite
from cdlib import algorithms
from cdlib import NodeClustering
import altair as alt
import sys
import os
sys.path.append("/Users/victoriagao/Documents/MSc/Schwartz_lab/altair-themes/")
if True:  # In order to bypass isort when saving
    import altairThemes

spot_diameter = 89.43 #pixels

Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'infomap', 'bayanpy', 'graph_tool'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'pyclustering', 'ASLPAw'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'infomap'}


In [3]:
def readCsv(x):
  """Parse file."""
  #colNames = ["method", "benchmark", "start", "end", "time", "memory"]
  df = pd.read_csv(x, sep=",")

  return df

def preprocessDf(df):
  """Transform ligand and receptor columns."""
  df["ligand-receptor"] = df["ligand"] + '-' + df["receptor"]
  df["component"] = df["component"] #.astype(str).str.zfill(2)

  return df

In [4]:
# Load gene_ids
gene_ids = []
with open("/Users/victoriagao/local_docs/NEST/stored_variables/gene_ids.txt", 'r') as file:
    for line in file:
        # Remove trailing newline characters and any leading/trailing whitespaces
        line = line.strip()
        gene_ids.append(line)

# Load coordinates
coordinates = np.load("/Users/victoriagao/local_docs/NEST/stored_variables/coordinates.npy")

# Load cell_barcode
with open('/Users/victoriagao/local_docs/NEST/stored_variables/cell_barcode.pkl', 'rb') as file:
    cell_barcode = pickle.load(file)

# Load connected_components (assume already have it)
with open("/Users/victoriagao/local_docs/NEST/stored_variables/filtered_connected_components.pkl", 'rb') as file:
    filtered_connected_components = pickle.load(file)

# Load pathologist's label
data_name = 'PDAC_64630'
if data_name == 'PDAC_64630':
    pathologist_label_file='/Users/victoriagao/local_docs/NEST/IX_annotation_artifacts_PDAC64630.csv' #IX_annotation_artifacts.csv' #
    pathologist_label=[]
    with open(pathologist_label_file) as file:
        csv_file = csv.reader(file, delimiter=",")
        for line in csv_file:
            pathologist_label.append(line)	
    	
    barcode_type=dict() # record the type (annotation) of each spot (barcode)
    for i in range (1, len(pathologist_label)):
        barcode_type[pathologist_label[i][0]] = pathologist_label[i][1]

# Load NEST output file into a 2D array
filename_str = 'NEST_combined_output_PDAC_64630'+'.csv'
inputFile = '/Users/victoriagao/local_docs/NEST/output/From_Fatema/'+filename_str
df = pd.read_csv(inputFile, sep=",")
csv_record_final = df.values.tolist()
df_column_names = list(df.columns)
csv_record_final = [df_column_names] + csv_record_final

# Load NEST output 
df = readCsv("/Users/victoriagao/local_docs/NEST/output/From_Fatema/NEST_combined_output_PDAC_64630.csv")
df_processed = preprocessDf(df)


In [5]:
# Get a list of unique LR pairs from the csv file
LR_pairs = df_processed['ligand-receptor'].unique()
len(LR_pairs)

147

In [12]:
#### i.e. making a list of Dictionary to store matched records for each region
matched_records_set_lib = {}  

for set_index, connected_comp_set in enumerate(filtered_connected_components):
    matched_records = []

    for record in csv_record_final:
        from_cell, to_cell = record[0], record[1]
        if (from_cell in connected_comp_set) ^ (to_cell in connected_comp_set): # must be tumor-stroma interacting
            matched_records.append(record)
    
    # Store matched records for this set index in the dictionary
    matched_records_set_lib[set_index+1] = matched_records

In [13]:
matched_records_set_lib

{1: [['GAATTTCTCGCTGCAG-1',
   'AACTTGCCCGTATGCA-1',
   'FN1',
   'RPSA',
   0.929611313343048,
   14,
   696,
   36],
  ['CAGTTCCGCGGGTCGA-1',
   'ATTCAGTAGCAGGGTC-1',
   'RPS19',
   'RPSA',
   0.9301169514656068,
   12,
   393,
   317],
  ['TAACGCTTTGAGAGCG-1',
   'CCGCACGTGACCTCGG-1',
   'FN1',
   'RPSA',
   0.929065489768982,
   14,
   1031,
   460],
  ['TCCCGGGTGTGCTGCT-1',
   'CCTCTCTCCCATCTAG-1',
   'PTPRF',
   'RACK1',
   0.9108615517616272,
   17,
   1157,
   492],
  ['GCAGCACACAGCCCAG-1',
   'CTTCGTAGATAGGTGA-1',
   'SERPINE1',
   'PLAUR',
   0.9101311564445496,
   18,
   774,
   653],
  ['AACTCAAGTTAATTGC-1',
   'GAAGTGATTTATCGTG-1',
   'COL1A1',
   'SDC1',
   0.9294158339500428,
   18,
   34,
   687],
  ['AACTTGCCCGTATGCA-1',
   'GAATTTCTCGCTGCAG-1',
   'THBS1',
   'SDC1',
   0.910299277305603,
   14,
   36,
   696],
  ['ACGTTAGATTTGCCCG-1',
   'GATTACTGAATTTGGG-1',
   'TIMP1',
   'LRP1',
   0.9112739443778992,
   29,
   141,
   759],
  ['GTTCACAGGAGTCTAG-1',
   'GCTGGCATAT

In [14]:
# Create count matrix for all 144 of LR pairs in each region
num_sets = len(matched_records_set_lib)
num_pairs = len(LR_pairs)
count_matrix = pd.DataFrame(index=range(1, num_sets + 1), columns=LR_pairs)

# Fill the count matrix
for set_number, records in matched_records_set_lib.items():
    for lr_pair in LR_pairs:
        count = sum(1 for record in records if f"{record[2]}-{record[3]}" == lr_pair)
        count_matrix.loc[set_number, lr_pair] = count

# # Print the count matrix
# print(count_matrix)

count_matrix_df = pd.DataFrame(count_matrix)

In [15]:
count_matrix_df

,LGALS3-NPTN,FN1-RPSA,PTPRF-RACK1,INS-HLA,TIMP1-LRP1,THBS2-LRP1,THBS1-SDC1,MIF-CD74,LGALS3-ITGB4,RPS19-RPSA,...,COL18A1-ITGB5,HSPG2-DAG1,PSEN1-DAG1,TGFB2-ACVR1B,ITGB1-DAG1,BMP4-BMPR2,BMP2-ENG,HMGB1-AGTRAP,TGFB1-SDC2,no-ligand-no-receptor
1,0,4,2,0,1,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,1,1,0,2,2,...,0,0,0,0,0,0,0,0,0,0
3,0,11,16,2,3,1,5,0,15,10,...,1,0,0,0,0,0,0,2,1,0
4,4,2,0,1,0,0,0,0,15,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,9,0,0,0,1,0,2,0,...,1,0,0,0,0,0,0,2,0,0
6,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,1,3,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
8,4,0,0,0,0,0,0,0,6,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# ### try out plotting a histogram using count_matrix_df
# df = pd.DataFrame(count_matrix_df)
# df.index.name = 'Region'  # Set the row index name to 'Region'

# # Reset the index to convert the row index into a column
# df.reset_index(inplace=True)

# # Melt the DataFrame to transform it into the desired format
# df = df.melt(id_vars='Region', var_name='L-R pair', value_name='Count')

# # Create a new DataFrame where each row represents a count in a region for a ligand-receptor pair
# output_df = pd.DataFrame(columns=['Region', 'L-R pair'])

# for index, row in df.iterrows():
#     region = row['Region']
#     l_r_pair = row['L-R pair']
#     count = row['Count']
#     rows_to_add = [{'Region': region, 'L-R pair': l_r_pair}] * count
#     output_df = pd.concat([output_df, pd.DataFrame(rows_to_add)], ignore_index=True)

# def plot(df):
#   # set1 = altairThemes.get_colour_scheme("Set1", len(df["Region"].unique()))
#   # set1[0] = '#000000'
#   base = alt.Chart(df).mark_bar().encode(
#             x=alt.X("L-R pair:N", axis=alt.Axis(labelAngle=45), sort='-y'),
#             y=alt.Y("count()"),
#             color=alt.Color("Region:N", scale =alt.Scale(scheme='category20')),
#             order=alt.Order("Region:N", sort="ascending"),
#             tooltip=["Region"]
#         )
#   p = base

#   return p

# p = plot(output_df)

# outPath = '/Users/victoriagao/local_docs/NEST/output/new__hist.html'
# p.save(outPath)


In [17]:
# Compiling all log odds calculationg into one big sheet
number_of_regions = len(filtered_connected_components)

# interested_region = 0
# comparing_region = 6

# Define a folder where you want to save the tables
output_folder = "/Users/victoriagao/local_docs/NEST/output/"

epsilon = 1e-10 

regions = count_matrix_df.columns.tolist()

# Create all possible combinations of 2 regions
region_combinations = itertools.combinations(regions, 2)
whole_log_odds_df = pd.DataFrame(LR_pairs, columns=["L-R Pair"])

for i in range(0,number_of_regions):
    print("region",i+1)
    for j in range(i + 1, number_of_regions):
        print("comparing with region",j+1)
        # region1 = regions[i]
        # region2 = regions[j]
        single_log_odds_df = pd.DataFrame()
        result_data = []

        # print(region1_data)

        for lr_pair in LR_pairs:
            
            
            # Create a DataFrame to store the contingency table
            columns = ['Region 1', 'Region 2']
            index = ['# of LR pair', '# of non LR pair']
            contingency_table = pd.DataFrame(index=index, columns=columns)
            contingency_table.fillna(0, inplace=True)  # Fill NaN cells with 0
            
            # # Calculate the sum of LR pairs other than lr_pair
            count_matrix_interested_row = count_matrix_df.iloc[i]
            count_matrix_comparing_row = count_matrix_df.iloc[j]
            
            # # Populate the contingency table
            contingency_table.loc['# of LR pair','Region 1'] = count_matrix_interested_row[lr_pair]
            contingency_table.loc['# of non LR pair','Region 1'] = count_matrix_interested_row.drop(lr_pair).sum()
            
            contingency_table.loc['# of LR pair','Region 2'] = count_matrix_comparing_row[lr_pair]
            contingency_table.loc['# of non LR pair','Region 2'] = count_matrix_comparing_row.drop(lr_pair).sum()

            # Extract the relevant counts from the contingency table
            Ad = contingency_table.loc['# of LR pair','Region 1']
            Bc = contingency_table.loc['# of LR pair','Region 2']
            Bd = contingency_table.loc['# of non LR pair','Region 1']
            Ac = contingency_table.loc['# of non LR pair','Region 2']

            # print(lr_pair, contingency_table)

            # Handle corner cases
            if Ad == 0 and Bc != 0:
                log_odds_ratio = -float('inf')  # Negative infinity
            elif Bc == 0 and Ad != 0:
                log_odds_ratio = float('inf')   # Positive infinity
            elif Bc == 0 and Ad == 0: # Cornor case: if the LR is not observed in neither region
                log_odds_ratio = None
            else:
                # odds_ratio = (Ad / Bc) / (Bd / Ac)
                # log_odds_ratio = np.log(odds_ratio)
                Bc = max(Bc, epsilon)
                Bd = max(Bd, epsilon)
                Ac = max(Ac, epsilon)
                odds_ratio = (Ad / Bc) / (Bd / Ac)
            
                if odds_ratio == 0:  # To handle the case where odds_ratio is very close to 0
                    log_odds_ratio = -float('inf')
                else:
                    log_odds_ratio = np.log(odds_ratio)
            
        # Store the result in the result_data list
            result_data.append({'L-R Pair': lr_pair, f"region{i+1} VS {j+1}": log_odds_ratio})
            # print(log_odds_ratio)
            
     # Create a DataFrame from the result_data list
        single_log_odds_df = pd.DataFrame(result_data)
        whole_log_odds_df = pd.merge(whole_log_odds_df, single_log_odds_df, on='L-R Pair', how='inner')

        # log_odds_df = log_odds_df.sort_values(by='log_odds_ratio', ascending=True)
        # log_odds_df['Ranking'] = log_odds_df['log_odds_ratio'].rank(ascending=False)

        # Save
        file_name = f"BIG_log_odds_for_TS_only.csv"
        table_path = os.path.join(output_folder, file_name)
        whole_log_odds_df.to_csv(table_path, index=False)

# # Create a DataFrame from the region_results list
# result_df = pd.DataFrame(region_results)

# # Print the results
# print(result_df)

region 1
comparing with region 2
comparing with region 3
comparing with region 4
comparing with region 5
comparing with region 6
comparing with region 7
comparing with region 8
comparing with region 9
comparing with region 10
comparing with region 11
comparing with region 12
comparing with region 13
comparing with region 14
comparing with region 15
comparing with region 16
comparing with region 17
comparing with region 18
comparing with region 19
comparing with region 20
comparing with region 21
region 2
comparing with region 3
comparing with region 4
comparing with region 5
comparing with region 6
comparing with region 7
comparing with region 8
comparing with region 9
comparing with region 10
comparing with region 11
comparing with region 12
comparing with region 13
comparing with region 14
comparing with region 15
comparing with region 16
comparing with region 17
comparing with region 18
comparing with region 19
comparing with region 20
comparing with region 21
region 3
comparing wit